## Exercise - Black’s Formula for Options on Futures

In [1]:
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
# Path to the Excel file containing SOFR futures option data
data_path_futures_options = "../data/option_data_bb_SFRZ5.xlsx"
data_path_historical_ref_rates = "../data/ref_rates.xlsx"

# Read the first sheet of the Excel file into a DataFrame
spot = (
    pd.read_excel(data_path_futures_options, sheet_name=0)
    .rename(columns={"Unnamed: 0": "field"})
    .set_index("field")
)

# Read the second sheet of the Excel file into a DataFrame
historical_ref_rates = pd.read_excel(data_path_historical_ref_rates, sheet_name=1)

# Read the sheet with the nearest expiration option data from the Excel file
nearest_expiration = pd.read_excel(
    data_path_futures_options,
    sheet_name=1
)

# Filter the DataFrame to get all call options ('C')
calls = nearest_expiration.loc[nearest_expiration["option type"] == "C"]

# Filter the DataFrame to get all put options ('P')
puts = nearest_expiration.loc[nearest_expiration["option type"] == "P"]

# Display the processed DataFrame
display(spot.style.set_caption("Underlying Spot"))
display(calls.head().style.set_caption("Nearest Expiration Calls"))
display(puts.head().style.set_caption("Nearest Expiration Puts"))
display(historical_ref_rates.head().style.set_caption("Historical Reference Rates"))

,SFRZ5 Comdty
field,
name,3 MONTH SOFR FUT Dec25
last update date,2025-07-29 00:00:00
last update time,10:03:48
price,96.085000
hist vol 30d,0.006851
hist vol 60d,0.007595
volume,150071


,ticker,last update date,last update time,days to expiration,option type,exercise style,option contract size,strike price,price,finance rate,time value,implied vol,delta,gamma,vega,theta,bid,ask,bid size,ask size,open int,volume
0,SFRV5C 95.75 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,C,AMER,2500,95.750000,0.330000,0.043960,-0.005000,0.140374,0.899011,0.028984,0.002962,-0.000265,0.335000,0.350000,51,51,10703,nan
1,SFRV5C 95.8125 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,C,AMER,2500,95.812500,0.277500,0.043960,0.005000,0.141465,0.847946,0.037543,0.003990,-0.000369,0.285000,0.295000,120,333,0,nan
2,SFRV5C 95.875 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,C,AMER,2500,95.875000,0.232500,0.043960,0.022500,0.150695,0.768768,0.045176,0.005212,-0.000524,0.235000,0.245000,14976,119,20020,nan
3,SFRV5C 95.9375 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,C,AMER,2500,95.937500,0.195000,0.043960,0.047500,0.162524,0.679044,0.049114,0.006161,-0.000676,0.195000,0.205000,15838,248,7013,nan
4,SFRV5C 96 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,C,AMER,2500,96.000000,0.160000,0.043960,0.075000,0.176306,0.589523,0.049138,0.006712,-0.000805,0.160000,0.175000,7500,19302,15635,nan


,ticker,last update date,last update time,days to expiration,option type,exercise style,option contract size,strike price,price,finance rate,time value,implied vol,delta,gamma,vega,theta,bid,ask,bid size,ask size,open int,volume
11,SFRV5P 95.75 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,P,AMER,2500,95.750000,0.007500,0.043960,0.007500,0.126942,-0.076812,0.025414,0.002511,-0.000220,0.005000,0.010000,25297,11208,57342,2550.000000
12,SFRV5P 95.8125 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,P,AMER,2500,95.812500,0.017500,0.043960,0.017500,0.135652,-0.138375,0.036404,0.003844,-0.000361,0.012500,0.020000,50328,39390,18597,nan
13,SFRV5P 95.875 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,P,AMER,2500,95.875000,0.035000,0.043960,0.035000,0.146440,-0.219462,0.045113,0.005143,-0.000522,0.027500,0.035000,40887,8311,50736,5225.000000
14,SFRV5P 95.9375 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,P,AMER,2500,95.937500,0.057500,0.043960,0.057500,0.161316,-0.313520,0.049041,0.006158,-0.000689,0.050000,0.060000,33397,49762,18828,nan
15,SFRV5P 96 Comdty,2025-07-28 00:00:00,2025-07-28 00:00:00,73,P,AMER,2500,96.000000,0.085000,0.043960,0.085000,0.168193,-0.398240,0.051125,0.006694,-0.000782,0.075000,0.085000,53974,758,62162,1000.000000


,date,DTB3,DFF,SOFR
0,2018-01-01 00:00:00,nan,1.330000,nan
1,2018-01-02 00:00:00,1.420000,1.420000,nan
2,2018-01-03 00:00:00,1.390000,1.420000,nan
3,2018-01-04 00:00:00,1.390000,1.420000,nan
4,2018-01-05 00:00:00,1.370000,1.420000,nan


## 1. SOFR Futures Options